In [57]:
import numpy as np

import astropy.io.fits as fits

import glob

In [58]:
dir = './fits_files/ck04models/ckm05/'
filepath = glob.glob(dir+'*.fits')

In [59]:
for filename in filepath:
    spectra_data = fits.getdata(filename)

In [60]:
# Progress!
for filename in filepath:
    spectrum = fits.open(filename)

In [61]:
!python ./stack/miniconda3-4.3.21-10a4fa6/DarwinX86/sims_photUtils/2.5.0.sims/python/lsst/sims/photUtils/Sed.py

Traceback (most recent call last):
  File "./stack/miniconda3-4.3.21-10a4fa6/DarwinX86/sims_photUtils/2.5.0.sims/python/lsst/sims/photUtils/Sed.py", line 98, in <module>
    from .PhysicalParameters import PhysicalParameters
ModuleNotFoundError: No module named '__main__.PhysicalParameters'; '__main__' is not a package
